In [689]:
%pip freeze > requirements.txt
# %pip install -r requirements.txt

Note: you may need to restart the kernel to use updated packages.


In [690]:
# from langchain.agents import AgentType
from langchain.agents import Tool
from langchain_core.tools import ToolException
# from langchain.tools import tool
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from langchain.agents import AgentExecutor, create_react_agent
from langchain.memory import ConversationBufferMemory
from langchain_community.utilities import SerpAPIWrapper
# from jinja2 import Template

from dotenv import load_dotenv
import json
import os
# import re
import json
import random

In [691]:
load_dotenv()
openai_api_key = os.getenv("OPENAI_API_KEY")

In [692]:
JSON_FILE_PATH = "..\\use_openai\\data\\id_added_dummy_data.json"
RCMD_PROMPT_PATH = "..\\prompts\\action_recommend.md"
FEATURE_ID_PATH = "featureId.txt"

In [693]:
# Read prompt
with open(RCMD_PROMPT_PATH, "r", encoding="utf-8", errors="replace") as f:
    RCMD_PROMPT = f.read()

# Read json data
with open(JSON_FILE_PATH, 'r', encoding='utf-8') as f:
    json_data = json.load(f)
    print(type(json_data))  # (test) <class 'list'>

# Read featureId keys
with open(FEATURE_ID_PATH, "r", encoding="utf-8") as f:
    lines = f.readlines()
    feature_id_list = [line.strip() for line in lines]
    print(feature_id_list)  # test

<class 'list'>
['26', '93', '36', '50', '16', '30', '63', '91', '99', '52', '94', '55', '13', '61', '14', '71', '35', '50', '52', '49', '91', '19', '79', '12', '70', '48', '48', '76', '53', '5', '88', '54', '98', '45', '61', '14', '93', '7', '58', '22', '86', '9', '59', '72', '52', '98', '67', '20', '31', '76', '62', '80', '45', '85', '67', '100', '80', '37', '11', '78', '96', '39', '65', '36', '49', '44', '42', '75', '28', '86', '43', '96', '57', '1', '39', '10', '67', '22', '54', '81', '11', '63', '55', '4', '75', '77', '95', '52', '42', '68', '24', '95', '26', '13', '43', '94', '64', '93', '87', '24', '91', '100', '11', '72', '49', '69', '37', '62', '72', '49', '3', '24', '3', '13', '93', '41', '9', '72', '38', '28', '45', '81', '15', '58', '75', '88', '23', '40', '12', '93', '62', '86', '6', '48', '45', '4', '91', '14', '89', '59', '81', '99', '35', '87', '88', '80', '56', '74', '4', '87', '79', '38', '95', '84', '91', '8', '78', '24', '77', '30', '79', '11', '21', '10', '30', '6',

In [694]:
# test
FEATURE_ID = int(random.choice(feature_id_list))
print(FEATURE_ID, type(FEATURE_ID))

24 <class 'int'>


In [695]:
# featureId로 해당 category, feature 이하의 actions만 추출
FEATURE_ID = int(random.choice(feature_id_list))

def extract_workspace_view(input_data, id):
    # featureId로 작업목록 + workspaceId 가져오기
    for item in input_data:
        for category in item.get("recommendedCategories", []):
            for feature in category.get("features", []):
                if feature.get("featureId") == id:
                    return {
                        "workspaceId": item["workspaceId"],
                        "recommendedCategories": [
                            {
                                "categoryId": category["categoryId"],
                                "name": category["name"],
                                "features": [feature]
                            }
                        ]
                    }

# featureId로 읽어온 actions
workspace_data = extract_workspace_view(json_data, FEATURE_ID)

if workspace_data:
    print(type(workspace_data))  # <class 'dict'>
    print(json.dumps(workspace_data, ensure_ascii=False, indent=2))
else:
    print("해당 featureId를 찾을 수 없습니다.")

<class 'dict'>
{
  "workspaceId": 15,
  "recommendedCategories": [
    {
      "categoryId": 4,
      "name": "게시판 기능",
      "features": [
        {
          "featureId": 47,
          "name": "관리 기능",
          "actions": [
            {
              "name": "백업/복구",
              "importance": 2,
              "startDate": "2025-06-19T14:25:34.095379",
              "endDate": "2025-07-03T14:25:34.095379"
            },
            {
              "name": "스팸 필터링",
              "importance": 3,
              "startDate": "2025-06-17T14:25:34.095382",
              "endDate": "2025-07-01T14:25:34.095382"
            }
          ]
        }
      ]
    }
  ]
}


In [696]:
# # featureId로 읽어온 actions
# workspace_data = extract_workspace_view(json_data, FEATURE_ID)

# # Jinja2 template 객체 생성. 템플릿 구조상(복잡, dict 생성이 필요) 변수 렌더링 필요.
# template = Template(RCMD_PROMPT)

# # 템플릿&작업목록 데이터 결합. jinja2 방식 템플릿 렌더링.
# # template 내 변수({})들을 실제값으로 치환 -> 문자열 생성
# formatted_input = template.render(ws_data=workspace_data)  # JSON 내 필드만 하나씩 꺼내서 출력
# formatted_input = template.render(ws_data=json.dumps(workspace_data, ensure_ascii=False))   #  JSON 전체를 문자열로 템플릿에 삽입

# print(formatted_input)  # test
# print(type(formatted_input))

In [697]:
# test
# search = DuckDuckGoSearchRun()
search = SerpAPIWrapper()
dump_data = json.dumps(workspace_data, ensure_ascii=False)  # dict -> str
# pattern = r'({[\s\S]*})'
# match = re.search(pattern, dump_data, re.DOTALL)
# json_block = match.group(0)
print(type(dump_data))
print(dump_data)

<class 'str'>
{"workspaceId": 15, "recommendedCategories": [{"categoryId": 4, "name": "게시판 기능", "features": [{"featureId": 47, "name": "관리 기능", "actions": [{"name": "백업/복구", "importance": 2, "startDate": "2025-06-19T14:25:34.095379", "endDate": "2025-07-03T14:25:34.095379"}, {"name": "스팸 필터링", "importance": 3, "startDate": "2025-06-17T14:25:34.095382", "endDate": "2025-07-01T14:25:34.095382"}]}]}]}


In [698]:
# OpenAI 객체 초기화
llm = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0.3,
    max_tokens=1024,
    openai_api_key=openai_api_key
)

In [699]:
RCMD_PROMPT

'당신은 JSON 기반의 워크스페이스 작업 흐름을 분석하여, 각 기능(feature)에 대해 적절한 후속 작업(action)을 제안하는 전문가입니다.\n\n당신의 임무는 다음과 같습니다:\n\n1. 입력으로 주어진 JSON에서 workspaceId, category, feature, action 구조를 파악하고,\n2. 각 feature에 대해 3개의 후속 작업(action)을 생성하며,\n3. 이 결과를 지정된 형식의 JSON으로 반환하는 것입니다.\n\n---\n\n입력 정보:\n\n- 입력 JSON: {input}\n- 사용 가능한 도구 목록: {tools}\n- 사용 가능한 도구 이름: {tool_names}\n- 이전 실행 이력: {agent_scratchpad}\n\n---\n\n아래의 ReAct 형식 규칙을 반드시 따르십시오. 하나라도 어기면 시스템이 응답을 인식하지 못하고 실패합니다.\n\nReAct 실행 구조:\n\n- Thought: 현재 해야 할 판단과 적절한 도구 선택 이유를 서술하십시오.\n- Action: 사용할 도구의 이름을 정확히 명시하십시오 (예: Action: ParseWorkflow). 반드시 tool_names에 있는 이름과 일치해야 하며, 대소문자도 구분됩니다.\n- Action Input: 선택한 도구에 전달할 입력 데이터를 JSON 형식으로 작성하십시오.\n   - 반드시 기존 actions 리스트는 유지하지 말고, 새롭게 생성한 actions 리스트로 완전히 덮어써야 합니다.\n   - 즉, feature["actions"] = 새로 생성된 리스트 방식으로 덮어쓰기 하십시오. 기존 action을 보존하거나 추가하지 마십시오.\n- Observation: 도구 실행 결과를 간단히 요약하십시오.\n- Final Answer: FinalAnswer 도구를 호출하려면 반드시 다음 형식을 따르십시오.\n    Final Answer: {{\n        "workspaceId": 1,\n        "recommendedC

In [700]:
# template

In [701]:
# ReAct template 생성
prompt = PromptTemplate(
    input_variables=["input", "tools", "tool_names", "agent_scratchpad"],
    template=RCMD_PROMPT
    # template=template
)

In [702]:
# llm(OpenAI) <-> Langchain 연결
llm_chain = LLMChain(prompt=prompt, llm=llm)

# LLM으로 action 추천 받는 함수
def generate_next_actions(s_result):
    return llm_chain.run({
        "input": s_result,
        "tools": "ParseWorkflow, FinalAnswer",
        "tool_names": "ParseWorkflow, FinalAnswer",
        "agent_scratchpad": ""
        })

# 전체 JSON을 받아 feature별 추천 작업을 생성하는 메인 함수
# @tool(description="워크스페이스 JSON으로부터 feature별 추천 작업을 생성합니다.")
def parse_and_generate_all(input_str: dict) -> str:

    # json_data = input_str  # 원본 데이터 보호용
    if isinstance(input_str, str):
        json_data = json.loads(input_str)
    else: json_data = input_str

    # 웹 검색 툴 정의
    search = SerpAPIWrapper()

    for category in json_data.get("recommendedCategories", []):  # recommendedCategories 가져오기
        for feature in category.get("features", []):        # recommendedCategories 내부의 feature에서

            # 웹 검색
            try:
                search_query = f"{feature['name']} 기능 구현 방법"  # feature name에 대한 actions 서치
                search_result = search.run(search_query)
            except Exception as e:
                search_result = f"** 웹 검색 실패: {e} **"
                
            # featureName, existingActions 변수 사용 안 하고 서치
            llm_input = search_result[:500]
            
            generated_actions = []

            # LLM 사용
            try:
                generated_json_str = generate_next_actions(llm_input)  # 추천 actions 생성
                generated_actions = json.loads(generated_json_str)     # 추천 actions를 json형식의 python 객체로 load
                # generated_actions = feature["actions"].extend(generated_actions)
                # feature["actions"] = generated_actions.copy()  # 덮어쓰기 방식

                # 생성된 action 리스트가 올바른 구조인지 검증
                if isinstance(generated_actions, list) and all("name" in a for a in generated_actions):
                    feature["actions"] = generated_actions.copy()

            except Exception as e:
                raise ToolException(f"ParseWorkflow 실행 중 오류 발생: {str(e)}")

    # return json.dumps(data, ensure_ascii=False, indent=2)
    return json_data

In [703]:
# parse_and_generate_all(workspace_data)  # test

In [704]:
# 툴 정의
# 파싱용(action 추천)
parse_tool = Tool(
    name="ParseWorkflow",
    func=parse_and_generate_all,
    description="JSON 작업 흐름에 대해 feature별로 다음 추천 작업(actions) 3개를 추가합니다."
)

# 출력용
final_tool = Tool(
    name="FinalAnswer",
    func=lambda x: x,  # 결과를 그대로 반환
    description="최종 JSON 결과를 반환합니다.",
    return_direct=True  # 도구 호출 즉시 결과 반환, agent 종료 
)

tools = [parse_tool, final_tool]
tool_names = [tool.name for tool in tools]

In [705]:
# 먼저 에이전트 생성하고
# create_react_agent()는 initialize_agent() 보다 세밀한 설정 가능
agent = create_react_agent(
    llm=llm,
    tools=tools,  # 단순 tools 선언, AgentExecutor에서 tools 재전달 필요
    prompt=prompt  # 프롬프트 여기서 넘김
)

# 메모리 설정
memory = ConversationBufferMemory(
    memory_key="agent_scratchpad",   # 대화 로그 저장용
    input_key="input"                # 사람이 말한 내용이 들어가는 키
)

# 다음에 AgentExecutor로 루프 반복 컨트롤?
agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,  # tools의 기능 전달
    # memory=memory,
    max_iterations=2,
    # max_execution_time=5,
    early_stopping_method="force",  # default="force"
    return_intermediate_steps=True,  # 중간 thought/action 추적
    handle_parsing_errors=True,
    verbose=True
)

In [706]:
# # tests
# prompt_filled = RCMD_PROMPT.replace("{input}", json.dumps(workspace_data, ensure_ascii=False))  # 위에서 이미 함
# prompt_filled

In [707]:
response = agent_executor.invoke({
    # "input": json.dumps(workspace_data, ensure_ascii=False),
    "input": workspace_data,
    # "tools": ["ParseWorkflow", "Final Answer"],
    "tool_names": tool_names,
    # "agent_scratchpad": ""
})

# 답변 생성 확인
print(response)



> Entering new AgentExecutor chain...
Thought: 주어진 JSON에서 각 feature에 대해 3개의 후속 작업을 생성해야 합니다. 이를 위해 ParseWorkflow 도구를 사용하여 새로운 actions 리스트를 생성하겠습니다. 

Action: ParseWorkflow  
Action Input: {
    "workspaceId": 15,
    "recommendedCategories": [
        {
            "categoryId": 4,
            "name": "게시판 기능",
            "features": [
                {
                    "featureId": 47,
                    "name": "관리 기능",
                    "actions": []
                }
            ]
        }
    ]
}  


ToolException: ParseWorkflow 실행 중 오류 발생: Expecting value: line 1 column 1 (char 0)

In [ ]:
print(type(response))
response

<class 'dict'>


{'input': {'workspaceId': 4,
  'recommendedCategories': [{'categoryId': 1,
    'name': '회원 관리 기능',
    'features': [{'featureId': 90,
      'name': '회원 가입',
      'actions': [{'name': '소셜 로그인 연동',
        'importance': 2,
        'startDate': '2025-06-14T14:25:34.095708',
        'endDate': '2025-06-27T14:25:34.095708'},
       {'name': '이메일 인증 로직 구현',
        'importance': 3,
        'startDate': '2025-06-16T14:25:34.095711',
        'endDate': '2025-06-22T14:25:34.095711'}]}]}]},
 'tool_names': ['ParseWorkflow', 'FinalAnswer'],
 'output': 'Agent stopped due to iteration limit or time limit.',
 'intermediate_steps': [(AgentAction(tool='ParseWorkflow', tool_input='{\n    "workspaceId": 4,\n    "recommendedCategories": [\n        {\n            "categoryId": 1,\n            "name": "회원 관리 기능",\n            "features": [\n                {\n                    "featureId": 90,\n                    "name": "회원 가입",\n                    "actions": []\n                }\n            ]\n    